# Clustering Jerárquico - RESUELTOS


---
Este cuaderno contiene ejercicios prácticos para consolidar los conceptos del Módulo Clustering Jerárquico

## Configuración Inicial

Ejecute esta celda antes de comenzar los ejercicios.

In [ ]:
# Bibliotecas necesarias
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Scipy: clustering jerárquico
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster, cophenet
from scipy.spatial.distance import pdist, squareform

# Scikit-learn
from sklearn.cluster import AgglomerativeClustering
from sklearn.datasets import make_blobs, make_moons, load_iris
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score, silhouette_samples
from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score

# Configuración de visualización
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 11

# Reproducibilidad
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

print("Configuración completada.")

---

## Ejercicio 1: Comprensión del Algoritmo Aglomerativo

### Contexto

Para comprender el funcionamiento del clustering jerárquico aglomerativo, es fundamental entender cómo se construye la matriz de linkage y cómo interpretar cada fusión.

### Datos

In [ ]:
# Dataset pequeño para análisis manual
X_ej1 = np.array([
    [1.0, 1.0],   # Punto 0
    [1.5, 1.5],   # Punto 1
    [5.0, 5.0],   # Punto 2
    [5.5, 5.5],   # Punto 3
    [3.0, 3.0],   # Punto 4
    [8.0, 1.0],   # Punto 5
    [8.5, 1.5]    # Punto 6
])

# Visualización
plt.figure(figsize=(8, 6))
plt.scatter(X_ej1[:, 0], X_ej1[:, 1], c='steelblue', s=100, edgecolors='w')
for i, (x, y) in enumerate(X_ej1):
    plt.annotate(f'P{i}', (x + 0.15, y + 0.15), fontsize=12, fontweight='bold')
plt.xlabel('Característica 1')
plt.ylabel('Característica 2')
plt.title('Dataset para Ejercicio 1')
plt.grid(True, alpha=0.3)
plt.show()

### Tarea 1.1: Cálculo de la Matriz de Distancias

Calcule la matriz de distancias euclidianas entre todos los pares de puntos.

In [ ]:
# TODO: Calcular la matriz de distancias
# Use scipy.spatial.distance.pdist y squareform

# Su código aquí
distancias_condensadas = pdist(X_ej1, metric='euclidean')  # Reemplazar
matriz_distancias = squareform(distancias_condensadas)  # Reemplazar

# Mostrar matriz de distancias
if matriz_distancias is not None:
    print("Matriz de distancias:")
    df_dist = pd.DataFrame(
        matriz_distancias,
        index=[f'P{i}' for i in range(len(X_ej1))],
        columns=[f'P{i}' for i in range(len(X_ej1))]
    )
    print(df_dist.round(3))

### Tarea 1.2: Identificación de la Primera Fusión

Identifique manualmente qué par de puntos tiene la menor distancia y por tanto se fusionará primero.

In [ ]:
# TODO: Encontrar el par de puntos con menor distancia
# Sugerencia: Use np.unravel_index y np.argmin sobre la matriz
# Recuerde que la diagonal es 0, debe ignorarla

# Su código aquí
# Crear copia y poner diagonal a infinito
matriz_temp = matriz_distancias.copy()
np.fill_diagonal(matriz_temp, np.inf)
min_idx = np.unravel_index(np.argmin(matriz_temp), matriz_temp.shape)
print(f"\nPar con menor distancia: P{min_idx[0]} y P{min_idx[1]}")
print(f"Distancia: {matriz_temp[min_idx]:.4f}")


# Verificar con la función linkage
Z_ej1 = linkage(X_ej1, method='single')
print("\nPrimera fusión según linkage (single):")
print(f"Clusters fusionados: {int(Z_ej1[0, 0])} y {int(Z_ej1[0, 1])}")
print(f"Distancia de fusión: {Z_ej1[0, 2]:.4f}")

### Tarea 1.3: Interpretación de la Matriz de Linkage

Analice la matriz de linkage completa y describa cada fusión.

In [ ]:
# TODO: Mostrar e interpretar la matriz de linkage completa
# Para cada fila, identificar qué clusters se fusionan
# Recuerde: índices >= n representan clusters formados previamente

print("Matriz de linkage (Single Linkage):")
print(f"{'Fusión':<8} {'Cluster A':<12} {'Cluster B':<12} {'Distancia':<12} {'Tamaño':<10}")
print("-" * 54)

n = len(X_ej1)
for i, fila in enumerate(Z_ej1):
    # TODO: Interpretar cada fusión
    # Si el índice >= n, es un cluster formado en la fusión (índice - n)
    a, b = int(fila[0]), int(fila[1])
    desc_a = f"P{a}" if a < n else f"C{a-n+1}"
    desc_b = f"P{b}" if b < n else f"C{b-n+1}"
    print(f"Fusión {i+1}: {desc_a} + {desc_b} -> C{i+1} (dist={fila[2]:.3f}, tam={int(fila[3])})")


### Pregunta de Reflexión 1

Observe la secuencia de fusiones:

1. ¿Qué grupos naturales se forman primero?
2. ¿Cuándo se fusiona el punto P4 (el punto central)?
3. ¿Cómo cambiaría la secuencia si usáramos complete linkage en lugar de single linkage?

*Escriba su respuesta aquí:*

...

---

## Ejercicio 2: Comparación de Criterios de Enlace

### Contexto

El criterio de enlace tiene un impacto significativo en la estructura jerárquica resultante. En este ejercicio, comparará los cuatro criterios principales.

### Datos

In [ ]:
# Dataset para comparación de criterios
X_ej2, y_ej2 = make_blobs(
    n_samples=100,
    n_features=2,
    centers=4,
    cluster_std=[0.8, 1.2, 0.6, 1.0],
    random_state=RANDOM_STATE
)

plt.figure(figsize=(8, 6))
plt.scatter(X_ej2[:, 0], X_ej2[:, 1], c='steelblue', alpha=0.6, edgecolors='w', s=50)
plt.xlabel('Característica 1')
plt.ylabel('Característica 2')
plt.title('Dataset para Ejercicio 2')
plt.show()

### Tarea 2.1: Calcular Linkage con Cada Criterio

Calcule la matriz de linkage para single, complete, average y ward.

In [ ]:
# TODO: Calcular linkage con cada criterio
metodos = ['single', 'complete', 'average', 'ward']
resultados_linkage = {}

for metodo in metodos:
    # Su código aquí
    resultados_linkage[metodo] = linkage(X_ej2, method=metodo)

# Verificar
for metodo, Z in resultados_linkage.items():
    print(f"{metodo}: matriz de forma {Z.shape}")

### Tarea 2.2: Visualizar Dendrogramas

Genere los dendrogramas para cada criterio de enlace en una figura con 4 subplots.

In [ ]:
# TODO: Crear figura con 4 dendrogramas
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
axes = axes.flatten()

for idx, metodo in enumerate(metodos):
    ax = axes[idx]

    # TODO: Generar dendrograma para cada método
    # Use truncate_mode='lastp', p=20 para visualización más clara
    dendrogram(resultados_linkage[metodo], ax=ax, truncate_mode='lastp', p=20,
               leaf_rotation=90, leaf_font_size=8)

    ax.set_title(f'{metodo.capitalize()} Linkage')
    ax.set_xlabel('Observaciones')
    ax.set_ylabel('Distancia')

plt.tight_layout()
plt.show()

### Tarea 2.3: Comparar Particiones con k=4

Corte cada dendrograma para obtener 4 clusters y compare las particiones resultantes.

In [ ]:
# TODO: Obtener particiones y calcular métricas
k = 4

print(f"Comparación de criterios de enlace (k={k}):")
print("=" * 60)
print(f"{'Criterio':<15} {'Silueta':<12} {'ARI':<12} {'NMI':<12}")
print("-" * 60)

for metodo in metodos:
    # TODO: Obtener labels con fcluster
    # TODO: Calcular silhouette_score, adjusted_rand_score, normalized_mutual_info_score
        # Su código aquí
    labels = fcluster(resultados_linkage[metodo], 4, criterion='maxclust')
    sil = silhouette_score(X_ej2, labels)
    ari = adjusted_rand_score(y_ej2, labels)
    nmi = normalized_mutual_info_score(y_ej2, labels)
    print(f"{metodo.capitalize():<15} {sil:<12.4f} {ari:<12.4f} {nmi:<12.4f}")

### Tarea 2.4: Correlación Cofenética

Calcule el coeficiente de correlación cofenética para cada método.

In [ ]:
# TODO: Calcular correlación cofenética para cada método
# Use scipy.cluster.hierarchy.cophenet

distancias_orig = pdist(X_ej2)

print("Correlación cofenética por criterio:")
print("-" * 40)

for metodo in metodos:
    # Su código aquí
    c, _ = cophenet(resultados_linkage[metodo], distancias_orig)
    print(f"{metodo.capitalize()}: {c:.4f}")

### Pregunta de Reflexión 2

Analice los resultados de la comparación:

1. ¿Qué criterio de enlace produce la mejor silueta? ¿Y el mejor ARI?
2. ¿Qué criterio tiene la mejor correlación cofenética? ¿Coincide con el de mejor silueta?
3. Observe las diferencias visuales entre los dendrogramas. ¿Cuál parece tener clusters más balanceados?

*Escriba su respuesta aquí:*

...

---

## Ejercicio 3: Selección del Número de Clusters

### Contexto

Una ventaja del clustering jerárquico es que permite explorar diferentes niveles de granularidad sin re-ejecutar el algoritmo. En este ejercicio, aplicará diferentes métodos para determinar el número óptimo de clusters.

### Datos

In [ ]:
# Dataset con estructura desconocida
np.random.seed(123)
X_ej3, y_ej3_oculto = make_blobs(
    n_samples=200,
    n_features=2,
    centers=5,
    cluster_std=[0.7, 0.9, 0.8, 1.0, 0.6],
    random_state=123
)

# Calcular linkage (Ward)
Z_ej3 = linkage(X_ej3, method='ward')

plt.figure(figsize=(8, 6))
plt.scatter(X_ej3[:, 0], X_ej3[:, 1], c='steelblue', alpha=0.6, edgecolors='w', s=50)
plt.xlabel('Característica 1')
plt.ylabel('Característica 2')
plt.title('Dataset para Ejercicio 3 (estructura desconocida)')
plt.show()

### Tarea 3.1: Análisis de Distancias de Fusión

Extraiga las distancias de fusión y calcule las diferencias entre fusiones consecutivas para identificar saltos.

In [ ]:
# TODO: Extraer distancias de fusión y calcular diferencias
# Las distancias están en la tercera columna de Z (índice 2)

distancias_fusion = Z_ej3[:, 2]  # Reemplazar
diferencias = np.diff(distancias_fusion)  # Reemplazar (use np.diff)

# TODO: Identificar los 5 mayores saltos y sus k correspondientes
# Su código aquí
indices_saltos = np.argsort(diferencias)[-5:][::-1]
print("Mayores saltos en distancia:")
for idx in indices_saltos:
    k = len(X_ej3) - idx - 1
    print(f"  Fusión {idx+1}: diferencia={diferencias[idx]:.3f}, k={k}")


# Visualización
if diferencias is not None:
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))

    # Gráfico de distancias
    axes[0].plot(range(1, len(distancias_fusion)+1), distancias_fusion, 'o-', markersize=2)
    axes[0].set_xlabel('Número de fusión')
    axes[0].set_ylabel('Distancia')
    axes[0].set_title('Distancias de fusión')

    # Gráfico de diferencias (últimas 20)
    ultimas = 20
    # Corregido: El rango del eje x debe coincidir con la longitud de diferencias[-ultimas:]
    axes[1].bar(range(len(diferencias) - ultimas, len(diferencias)), diferencias[-ultimas:], color='steelblue')
    axes[1].set_xlabel('Número de fusión')
    axes[1].set_ylabel('Incremento en distancia')
    axes[1].set_title(f'Diferencias entre fusiones (últimas {ultimas})')

    plt.tight_layout()
    plt.show()

### Tarea 3.2: Método de la Silueta

Calcule el coeficiente de silueta para diferentes valores de k.

In [ ]:
# TODO: Calcular silueta para k = 2 a 10
k_range = range(2, 11)
siluetas = []

for k in k_range:
    # TODO: Obtener labels con fcluster y calcular silueta
    # Su código aquí
    labels = fcluster(Z_ej3, k, criterion='maxclust')
    siluetas.append(silhouette_score(X_ej3, labels))
    k_opt = list(k_range)[np.argmax(siluetas)]

# Visualización
if len(siluetas) > 0:
    plt.figure(figsize=(10, 5))
    plt.plot(k_range, siluetas, 'o-', linewidth=2, markersize=8)
    plt.axvline(x=k_opt, color='red', linestyle='--', label=f'k óptimo = {k_opt}')
    plt.xlabel('Número de clusters (k)')
    plt.ylabel('Coeficiente de Silueta')
    plt.title('Selección de k por Silueta')
    plt.legend()
    plt.xticks(k_range)
    plt.show()

### Tarea 3.3: Visualización del Dendrograma con Corte

Genere el dendrograma y marque el corte óptimo.

In [ ]:
# TODO: Generar dendrograma con línea de corte
fig, ax = plt.subplots(figsize=(14, 7))

# TODO: Crear dendrograma con dendrogram()
# TODO: Añadir línea horizontal en la altura de corte apropiada

# Su código aquí
dendrogram(Z_ej3, ax=ax, truncate_mode='lastp', p=30, color_threshold=15)
ax.axhline(y=25, color='red', linestyle='--', linewidth=2, label=f'Corte para k={k_opt}')
ax.legend()

ax.set_xlabel('Observaciones')
ax.set_ylabel('Distancia')
ax.set_title('Dendrograma con línea de corte')

plt.tight_layout()
plt.show()

### Pregunta de Reflexión 3

Basándose en los análisis realizados:

1. ¿Qué valor de k sugieren las diferencias en distancias de fusión?
2. ¿Qué valor de k maximiza el coeficiente de silueta?
3. ¿Los dos métodos coinciden? Si no, ¿cuál consideraría más confiable y por qué?

*Escriba su respuesta aquí:*

...

---

## Ejercicio 4: Efecto Cadena y Formas No Convexas

### Contexto

Single linkage puede detectar clusters de forma arbitraria, pero es susceptible al efecto cadena. En este ejercicio, explorará ambos comportamientos.

### Datos

In [ ]:
# Dataset 1: Lunas (formas no convexas)
X_moons, y_moons = make_moons(n_samples=200, noise=0.05, random_state=RANDOM_STATE)

# Dataset 2: Clusters con puente (propenso a efecto cadena)
np.random.seed(RANDOM_STATE)
cluster_a = np.random.randn(60, 2) * 0.4 + np.array([-2, 0])
cluster_b = np.random.randn(60, 2) * 0.4 + np.array([2, 0])
puente = np.column_stack([np.linspace(-1.5, 1.5, 8), np.random.randn(8) * 0.05])
X_puente = np.vstack([cluster_a, cluster_b, puente])

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].scatter(X_moons[:, 0], X_moons[:, 1], c=y_moons, cmap='viridis', edgecolors='w', s=50)
axes[0].set_title('Dataset 1: Lunas')
axes[0].set_xlabel('Característica 1')
axes[0].set_ylabel('Característica 2')

axes[1].scatter(cluster_a[:, 0], cluster_a[:, 1], c='#3498db', edgecolors='w', s=50, label='Cluster A')
axes[1].scatter(cluster_b[:, 0], cluster_b[:, 1], c='#e74c3c', edgecolors='w', s=50, label='Cluster B')
axes[1].scatter(puente[:, 0], puente[:, 1], c='#2ecc71', edgecolors='w', s=80, marker='s', label='Puente')
axes[1].set_title('Dataset 2: Clusters con puente')
axes[1].set_xlabel('Característica 1')
axes[1].set_ylabel('Característica 2')
axes[1].legend()

plt.tight_layout()
plt.show()

### Tarea 4.1: Clustering de Lunas con Diferentes Criterios

Aplique single, complete y ward linkage al dataset de lunas y compare los resultados.

In [ ]:
# TODO: Aplicar cada criterio y visualizar resultados
criterios = ['single', 'complete', 'ward']

fig, axes = plt.subplots(1, 3, figsize=(15, 5))

for idx, criterio in enumerate(criterios):
    ax = axes[idx]

    # TODO: Aplicar AgglomerativeClustering con n_clusters=2
    # TODO: Calcular ARI comparando con y_moons
    # TODO: Visualizar el resultado

    # Su código aquí
    modelo = AgglomerativeClustering(n_clusters=2, linkage=criterio)
    labels = modelo.fit_predict(X_moons)
    ari = adjusted_rand_score(y_moons, labels)

    ax.scatter(X_moons[:, 0], X_moons[:, 1], c=labels, cmap='viridis', edgecolors='w', s=50)
    ax.set_title(f'{criterio.capitalize()} (ARI={ari:.3f})')

    ax.set_xlabel('Característica 1')
    ax.set_ylabel('Característica 2')

plt.suptitle('Clustering de Lunas por Criterio de Enlace', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

### Tarea 4.2: Demostración del Efecto Cadena

Aplique single y ward linkage al dataset con puente y observe el efecto cadena.

In [ ]:
# TODO: Comparar single vs ward en el dataset con puente
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

for idx, criterio in enumerate(['single', 'ward']):
    ax = axes[idx]

    # TODO: Aplicar AgglomerativeClustering con n_clusters=2
    # TODO: Visualizar resultado
    # TODO: Contar cuántos puntos hay en cada cluster

    # Su código aquí
    modelo = AgglomerativeClustering(n_clusters=2, linkage=criterio)
    labels = modelo.fit_predict(X_puente)

    ax.scatter(X_puente[:, 0], X_puente[:, 1], c=labels, cmap='coolwarm', edgecolors='w', s=50)
    ax.set_title(f'{criterio.capitalize()} - Distribución: {np.bincount(labels)}')

    #ax.set_title(f'{criterio.capitalize()} Linkage')
    ax.set_xlabel('Característica 1')
    ax.set_ylabel('Característica 2')

plt.suptitle('Efecto Cadena: Single vs Ward', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

### Pregunta de Reflexión 4

Analice los resultados:

1. ¿Por qué single linkage funciona bien con las lunas pero falla con el puente?
2. ¿Cómo se podría mitigar el efecto cadena en single linkage?
3. ¿En qué situaciones reales podría ocurrir un "puente" de puntos entre clusters?

*Escriba su respuesta aquí:*

...

---

## Ejercicio 5: Caso Práctico - Segmentación de Especies

### Contexto

El dataset Iris es un clásico en machine learning. Aplicará clustering jerárquico para descubrir agrupaciones naturales de especies de flores.

### Datos

In [ ]:
# Cargar dataset Iris
from sklearn.datasets import load_iris

iris = load_iris()
X_iris = iris.data
y_iris = iris.target
nombres_features = iris.feature_names
nombres_especies = iris.target_names

print(f"Dimensiones: {X_iris.shape}")
print(f"Características: {nombres_features}")
print(f"Especies: {nombres_especies}")
print(f"\nDistribución de especies: {np.bincount(y_iris)}")

### Tarea 5.1: Preprocesamiento y Análisis Exploratorio

In [ ]:
# TODO: Estandarizar los datos
scaler = StandardScaler()
X_iris_scaled = scaler.fit_transform(X_iris)  # Reemplazar

# TODO: Visualizar pares de características coloreadas por especie real
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Su código aquí para crear scatter plots de:
# - sepal length vs sepal width
# - petal length vs petal width
# - sepal length vs petal length
pares = [(0, 1), (2, 3), (0, 2)]
for ax, (i, j) in zip(axes, pares):
    scatter = ax.scatter(X_iris[:, i], X_iris[:, j], c=y_iris, cmap='viridis', edgecolors='w', s=50)
    ax.set_xlabel(nombres_features[i])
    ax.set_ylabel(nombres_features[j])

plt.tight_layout()
plt.show()

### Tarea 5.2: Dendrograma y Selección de Clusters

In [ ]:
# TODO: Calcular linkage (Ward) y generar dendrograma
# TODO: Analizar la estructura del dendrograma

fig, ax = plt.subplots(figsize=(14, 7))

# Su código aquí
Z_iris = linkage(X_iris_scaled, method='ward')
dendrogram(Z_iris, ax=ax, truncate_mode='lastp', p=30, color_threshold=7)

ax.set_xlabel('Muestras')
ax.set_ylabel('Distancia')
ax.set_title('Dendrograma del Dataset Iris (Ward)')

plt.tight_layout()
plt.show()

### Tarea 5.3: Evaluación de la Partición

In [ ]:
# TODO: Evaluar particiones con k=2, 3 y 4
# TODO: Calcular silueta y ARI para cada k
# TODO: Determinar cuál k es más apropiado

print("Evaluación de diferentes valores de k:")
print("=" * 50)
print(f"{'k':<5} {'Silueta':<12} {'ARI':<12} {'NMI':<12}")
print("-" * 50)

for k in [2, 3, 4]:
    # Su código aquí
    labels = fcluster(Z_iris, k, criterion='maxclust')
    sil = silhouette_score(X_iris_scaled, labels)
    ari = adjusted_rand_score(y_iris, labels)
    nmi = normalized_mutual_info_score(y_iris, labels)
    print(f"k={k}: Silueta={sil:.3f}, ARI={ari:.3f}, NMI={nmi:.3f}")

### Tarea 5.4: Análisis de Errores de Clasificación

In [ ]:
# TODO: Con k=3, analizar la matriz de confusión entre clusters y especies
# TODO: Identificar qué especies se confunden más

# Sugerencia: Use pd.crosstab para crear matriz de contingencia
labels_k3 = fcluster(Z_iris, 3, criterion='maxclust') - 1
confusion = pd.crosstab(
    pd.Series(y_iris, name='Especie'),
    pd.Series(labels_k3, name='Cluster')
)
confusion.index = nombres_especies
print("\nMatriz de contingencia:")
print(confusion)

### Pregunta de Reflexión 5

Analice los resultados del clustering sobre Iris:

1. ¿Cuántos clusters naturales sugiere el dendrograma?
2. Con k=3, ¿qué especie es más fácil de separar? ¿Cuáles se confunden?
3. ¿Por qué la silueta para k=2 es mayor que para k=3, a pesar de que hay 3 especies?

*Escriba su respuesta aquí:*

...

---

## Ejercicio 6: Análisis de Escalabilidad

### Contexto

El clustering jerárquico tiene limitaciones de escalabilidad debido a su complejidad O(n² log n) temporal y O(n²) espacial. En este ejercicio, medirá empíricamente estos límites.

### Datos

In [ ]:
import time

# Tamaños de dataset a evaluar
tamanos = [100, 250, 500, 1000, 2000, 3000, 10000]

### Tarea 6.1: Medir Tiempos de Ejecución

In [ ]:
# TODO: Medir tiempo de ejecución para cada tamaño
tiempos = []

print(f"{'n':<10} {'Tiempo (s)':<15}")
print("-" * 25)

for n in tamanos:
    # TODO: Generar datos con make_blobs
    # TODO: Medir tiempo de linkage (Ward)

    # Su código aquí
    X_temp, _ = make_blobs(n_samples=n, n_features=2, centers=4, random_state=42)
    inicio = time.time()
    Z_temp = linkage(X_temp, method='ward')
    t = time.time() - inicio
    tiempos.append(t)
    print(f"{n:<10} {t:<15.4f}")

# TODO: Visualizar la relación entre n y tiempo
if len(tiempos) > 0:
    plt.figure(figsize=(10, 5))
    plt.plot(tamanos, tiempos, 'o-', linewidth=2, markersize=8)
    plt.xlabel('Número de observaciones (n)')
    plt.ylabel('Tiempo (segundos)')
    plt.title('Escalabilidad del Clustering Jerárquico')
    plt.show()

### Tarea 6.2: Estimar Complejidad Empírica

In [ ]:
# TODO: Ajustar un modelo para estimar la complejidad
# La complejidad teórica es O(n² log n)
# Verificar si los tiempos medidos siguen esta tendencia

# Sugerencia: Calcular t / (n² * log(n)) para cada n
# Si la complejidad es correcta, este ratio debería ser aproximadamente constante

if len(tiempos) > 0:
    print("Verificación de complejidad O(n² log n):")
    print(f"{'n':<10} {'t (s)':<12} {'t / (n² log n)':<20}")
    print("-" * 42)

    for n, t in zip(tamanos, tiempos):
        # Su código aquí
        ratio = t / (n**2 * np.log(n))
        print(f"{n:<10}{t:<12.4f}{ratio:<20.2e}")

### Pregunta de Reflexión 6

Basándose en sus mediciones:

1. ¿Aproximadamente cuánto tiempo tomaría procesar un dataset de 50,000 observaciones?
2. ¿Cuál sería el límite práctico de n en un entorno con restricciones de tiempo (ej. < 1 minuto)?
3. ¿Qué alternativas existen para aplicar clustering jerárquico a datasets grandes?

*Escriba su respuesta aquí:*

...

---
# EOF (End Of File)